# CD4+ T cell behavior in COVID-19 infection

In [1]:
import pandas as pd

env_cov = ['covmod', 'covsev', 'covtgfb', 'covil10']
ana_cov = ['cfm','mut']

data = []
for ana in ana_cov:
    for env in env_cov:
        filename = 'results/thsocs_{}_{}.csv'.format(ana,env)
        print(filename)
        df = pd.read_csv(filename)
        df['analysis'] = ana
        df['environment'] = env
        data.append(df)
data = pd.concat(data)
data.tail()

results/thsocs_cfm_covmod.csv
results/thsocs_cfm_covsev.csv
results/thsocs_cfm_covtgfb.csv
results/thsocs_cfm_covil10.csv
results/thsocs_mut_covmod.csv
results/thsocs_mut_covsev.csv
results/thsocs_mut_covtgfb.csv
results/thsocs_mut_covil10.csv


,initial,final,genes,values,analysis,environment
247,Tr,Tr,TGFBe,1,mut,covil10
248,Tr,Treg,FOXP3,1,mut,covil10
249,Tr,Treg,FOXP3,1,mut,covil10
250,Tr,Treg,FOXP3,1,mut,covil10
251,Tr,Treg,IL2,1,mut,covil10


In [2]:

#diff =  data[ (data['initial']!=data['final']) ]
diff = pd.DataFrame()
diff['Intervention'] = data['genes'] + '=' + data['values'].astype(str)
diff['Transition'] = data['initial'] + '→' + data['final']
diff['Perturb time'] = data['analysis'].replace({'cfm':'t=1','mut':'t=∞'})
diff['Environment'] = data['environment']
diff['Filter'] = data['final'].map({'Th1':'ΔTh1'}).fillna('Other')
diff = diff[data['initial']!=data['final']].drop_duplicates()
diff.tail()

,Intervention,Transition,Perturb time,Environment,Filter
206,GATA3=1,Tr→Th2R,t=∞,covil10,Other
209,IL12e=0,Tr→Th2R,t=∞,covil10,Other
210,IL4=1,Tr→Th2R,t=∞,covil10,Other
248,FOXP3=1,Tr→Treg,t=∞,covil10,Other
251,IL2=1,Tr→Treg,t=∞,covil10,Other


## Interventions that lead towards Th1

Ignore covmod, focus in severe covid

In [3]:
interventions =  diff[(diff['Filter']=='ΔTh1') & \
                      (diff['Environment']!='covmod')]
interventions = interventions.drop_duplicates()
interventions

,Intervention,Transition,Perturb time,Environment,Filter
76,IFNG=1,Tex→Th1,t=1,covsev,ΔTh1
79,SOCS1=0,Tex→Th1,t=1,covsev,ΔTh1
139,IL10=0,Th1R→Th1,t=1,covsev,ΔTh1
141,IL6=1,Th1R→Th1,t=1,covsev,ΔTh1
142,SOCS3=0,Th1R→Th1,t=1,covsev,ΔTh1
143,TGFB=0,Th1R→Th1,t=1,covsev,ΔTh1
61,IFNG=1,Tex→Th1,t=∞,covsev,ΔTh1
64,SOCS1=0,Tex→Th1,t=∞,covsev,ΔTh1
139,IL10=0,Th1R→Th1,t=∞,covsev,ΔTh1
141,TGFB=0,Th1R→Th1,t=∞,covsev,ΔTh1


In [4]:
interventions = interventions['Intervention'].unique()
interventions

array(['IFNG=1', 'SOCS1=0', 'IL10=0', 'IL6=1', 'SOCS3=0', 'TGFB=0'],
      dtype=object)

## Secondary effects

In [5]:
effect = diff[(diff['Intervention'].isin(interventions))]
effect = pd.pivot_table(effect, index=['Intervention','Filter'],columns=['Environment','Perturb time'],
                        values='Transition', aggfunc=lambda x:'\n'.join(x), fill_value='-')
effect

Environment                            covil10                                 \
Perturb time                               t=1                            t=∞   
Intervention Filter                                                             
IFNG=1       Other          Th2R→Th1R\nTr→Th1R  Naive→Tex\nTh2R→Th1R\nTr→Th1R   
             ΔTh1                            -                              -   
IL10=0       Other   Th1R→Tr\nTh2R→Tr\nTr→Th1R     Th1R→Tex\nTh2R→Tex\nTr→Tex   
             ΔTh1                            -                       Th1R→Th1   
IL6=1        Other            Th1R→Tr\nTh2R→Tr    Naive→Tex\nTh1R→Tr\nTh2R→Tr   
             ΔTh1                            -                              -   
SOCS1=0      Other                           -                      Naive→Tex   
             ΔTh1                            -                              -   
SOCS3=0      Other            Th1R→Tr\nTh2R→Tr               Th1R→Tr\nTh2R→Tr   
             ΔTh1                            -                              -   
TGFB=0       Other                           -                              -   
             ΔTh1                            -                              -   

Environment                              covmod  \
Perturb time                                t=1   
Intervention Filter                               
IFNG=1       Other           Th2R→Th1R\nTr→Th1R   
             ΔTh1                       Tex→Th1   
IL10=0       Other            Th2R→Tex\nTr→Th1R   
             ΔTh1            Th1R→Th1\nTh2R→Th1   
IL6=1        Other                     Th2R→Tex   
             ΔTh1    Th1R→Th1\nTh2R→Th1\nTr→Th1   
SOCS1=0      Other                            -   
             ΔTh1                       Tex→Th1   
SOCS3=0      Other           Th2R→Th1R\nTr→Th1R   
             ΔTh1                             -   
TGFB=0       Other                            -   
             ΔTh1              Th1R→Th1\nTr→Th1   

Environment                                                          \
Perturb time                                                    t=∞   
Intervention Filter                                                   
IFNG=1       Other                    Naive→Tex\nTh2R→Th1R\nTr→Th1R   
             ΔTh1                                           Tex→Th1   
IL10=0       Other                                Th2R→Tex\nTr→Th1R   
             ΔTh1                                Th1R→Th1\nTh2R→Th1   
IL6=1        Other   Naive→Tex\nTh1→Tex\nTh1R→Tex\nTh1R→Tr\nTh2R→Tr   
             ΔTh1                                                 -   
SOCS1=0      Other                                                -   
             ΔTh1                                           Tex→Th1   
SOCS3=0      Other                               Th2R→Th1R\nTr→Th1R   
             ΔTh1                                                 -   
TGFB=0       Other                                                -   
             ΔTh1                                  Th1R→Th1\nTr→Th1   

Environment                              covsev  \
Perturb time                                t=1   
Intervention Filter                               
IFNG=1       Other           Th2R→Th1R\nTr→Th1R   
             ΔTh1                       Tex→Th1   
IL10=0       Other   Th1R→Tex\nTh2R→Tex\nTr→Tex   
             ΔTh1                      Th1R→Th1   
IL6=1        Other   Th1R→Tex\nTh2R→Tex\nTr→Tex   
             ΔTh1                      Th1R→Th1   
SOCS1=0      Other                            -   
             ΔTh1                       Tex→Th1   
SOCS3=0      Other   Th1R→Tex\nTh2R→Tex\nTr→Tex   
             ΔTh1                      Th1R→Th1   
TGFB=0       Other             Th1R→Tex\nTr→Tex   
             ΔTh1                      Th1R→Th1   

Environment                                                          \
Perturb time                                                    t=∞   
Intervention Filter                         

In [6]:
effect.to_csv('results/thsocs_interventions.csv')